In [2]:
import numpy as np
import random 
from tqdm.auto import tqdm
import heapq

c:\Users\anjal\Desktop\computational Interlligence\CI2024_lab1\CI2024_lab1\CI2024_lab1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generation of the random matrix of N dimension

In [3]:
#just a function to check if the randomly generated matrix is solvable or not
def is_solvable(puzzle, n):
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(1 for i in range(len(flattened)) for j in range(i+1, len(flattened)) if flattened[i]>flattened[j])
    if n%2 != 0:
        return inversions % 2 == 0
    else:
        blank_row = n -np.where(puzzle == 0)[0][0]
        return (blank_row % 2 == 0) != (inversions %2 == 0)                                  

In [4]:
def matrix_generator(n):
    while True:
        puzzle = np.random.permutation(n**2).reshape(n,n)
        if is_solvable(puzzle, n):
            return puzzle

## Implementaion of A*

In [5]:
#the helper function used to calculate the Manhattan Distance heuristic

def manhattan_distance(state, goal):
    n = state.shape[0] #it is the length of state as Dimension of the matrix
    distance = 0
    for i in range(n):
        for j in range(n):
            if state[i,j] != 0:
                goal_pos = divmod(goal.index(state[i,j]), n)
                # goal_pos = np.argwhere(goal == state[i,j])[0]
                distance += abs(i-goal_pos[0]) + abs(j-goal_pos[1])
        return distance

In [6]:
#generation of neighbouring states by moving the blank tilt

def get_neighbors(state):
    n = state.shape[0]
    neighbors = []
    x, y = np.argwhere(state == 0)[0]
    moves = [(-1, 0), (1,0), (0, -1), (0,1)] #top , bottom, Left, right
    for dx, dy in moves:
        nx, ny = x+dx, y+dy
        if 0<=nx <n and 0 <= ny <n:
            new_state = state.copy()
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x,y]
            neighbors.append(new_state)
    return neighbors

In [7]:
def a_star_search(start, goal):
    n = start.shape[0]
    goal_flat = [tile for row in goal for tile in row]  # Flatten goal for heuristic
    
    # Normalize to Python tuple of integers
    start_tuple = tuple(tuple(int(val) for val in row) for row in start)
    goal_tuple = tuple(tuple(int(val) for val in row) for row in goal)

    # Priority queue for A* (f(n), g(n), state, path)
    frontier = [(manhattan_distance(start, goal_flat), 0, start_tuple, [])]
    heapq.heapify(frontier)
    visited = set()

    cost = 0  # Nodes evaluated

    while frontier:
        f, g, current, path = heapq.heappop(frontier)
        cost += 1

        # Convert `current` back to a NumPy array for processing
        current_state = np.array(current)

        # Debugging: Print the current state and frontier
        # print(f"Current State:\n{current_state}")
        # print(f"Frontier: {frontier}")

        # Check if the goal state is reached
        if np.array_equal(current_state, goal):
            return path, cost

        # Add the current state to visited (converted to tuple for hashability)
        visited.add(current)

        # Generate neighbors
        for neighbor in get_neighbors(current_state):
            # Normalize the neighbor to Python `int` tuples
            neighbor_tuple = tuple(tuple(int(val) for val in row) for row in neighbor)

            # Only add neighbor if not already visited
            if neighbor_tuple not in visited:
                h = manhattan_distance(neighbor, goal_flat)

                # Push to the frontier, ensuring everything is tuples and integers
                heapq.heappush(frontier, (
                    g + 1 + h,
                    g + 1,
                    neighbor_tuple,
                    path + [neighbor_tuple]  # Use tuples here as well
                ))

    return None, cost  # No solution found


In [8]:
def ida_star_search(start, goal):
    # Convert `goal` to a list for compatibility with `manhattan_distance`
    goal_flat = [int(tile) for row in goal for tile in row]

    def search(state, g, threshold, path, cost):
        f = g + manhattan_distance(state, goal_flat)

        # If the cost exceeds the threshold, return f-value
        if f > threshold:
            return f, None

        # If goal state is reached
        if np.array_equal(state, goal):
            return f, path

        min_threshold = float('inf')
        for neighbor in get_neighbors(state):
            # Avoid revisiting states in the current path
            if any(np.array_equal(neighbor, p) for p in path):
                continue

            # Continue search recursively
            new_path = path + [neighbor]
            cost += 1  # Increment cost for each node explored
            temp_threshold, result_path = search(neighbor, g + 1, threshold, new_path, cost)

            if result_path is not None:
                return temp_threshold, result_path

            min_threshold = min(min_threshold, temp_threshold)

        return min_threshold, None

    # Initial threshold is the heuristic value of the start state
    threshold = manhattan_distance(start, goal_flat)
    cost = 0  # Nodes evaluated counter

    while True:
        result_threshold, result_path = search(start, 0, threshold, [start], cost)

        if result_path is not None:
            quality = len(result_path) - 1  # Number of moves in the solution path
            cost = result_threshold
            efficiency = quality / cost if cost != 0 else 0
            return result_path, quality, cost, efficiency

        if result_threshold == float('inf'):
            return None, 0, 0, 0  # No solution found

        threshold = result_threshold  # Update threshold and repeat

In [ ]:
PUZZLE_DIM = 4
start = matrix_generator(PUZZLE_DIM)
goal = np.arange(1, PUZZLE_DIM**2).tolist() + [0]
goal = np.array(goal).reshape(PUZZLE_DIM,PUZZLE_DIM)

print("initial matrix is: ")
print(start)
print("the goal matrix is: ")
print(goal)

# solution_path, total_cost = a_star_search(start, goal)


# # print("\nSolution:")
# # for step in solution_path:
# #     print(np.array(step), "\n")
# print("Output using A*")
# print(f"Quality (Number of Moves): {len(solution_path)}") #length of the solution path in case of A* it is equivalent to the length of the solution path.
# print(f"Cost (Nodes Evaluated): {total_cost}") #total no of state the algorithms has evaluated or processed in order to find the solution.
# print(f"Efficiency (Quality / Cost): {len(solution_path) / total_cost:.4f}")


solution, quality, cost, efficiency = ida_star_search(start, goal)
print("output of IDA* with heuristic")
print(f"Quality (Number of Moves): {quality}")
print(f"Cost (Nodes Evaluated): {cost}")
print(f"Efficiency (Quality / Cost): {efficiency:.4f}")

initial matrix is: 
[[ 8 10 15  9]
 [ 6  2  4  5]
 [14  3 11  7]
 [12 13  0  1]]
the goal matrix is: 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
